# Project Title
### Data Engineering Capstone Project

#### Project Summary
The goal of this project is to create a data lake with data about immigration in EEUU, which facilitates the analysis and predictions to several types of company.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

import pyreadstat

import configparser
import os

import pandas as pd
import numpy as np

output_data = "s3a://bucket-test-udacity/"

In [42]:
pip install fsspec

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
pip install s3fs

     ---------------------------------------- 66.8/66.8 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.     ------------------------------------- 323.6/323.6 kB 10.1 MB/s eta 0:00:00




[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 9.1/9.1 MB 20.0 MB/s eta 0:00:00
     ---------------------------------------- 56.8/56.8 kB 2.9 MB/s eta 0:00:00


In [2]:
def create_spark_session():
    spark = SparkSession.builder.config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0").getOrCreate()
    return spark
spark = create_spark_session()

In [3]:
spark

### Step 1: Scope the Project and Gather Data

#### SCOPE 

The purpose of this project is to create a data lake with data about immigration in the US and the circumstances in which it has occurred.

To carry it out, we are considering three datasets, which are going to be raw data for our data lake.

**The datasets used are:**
- **I94 Immigration Data:** This data comes from the US National Tourism and Trade Office. A data dictionary is included in the workspace.  **[This](https://www.trade.gov/i-94-arrivals-program)** is where the data comes from. The National Travel and Tourism Office (NTTO) manages the ADIS/I-94 visitor arrivals program in cooperation with the Department of Homeland Security (DHS)/U.S. Customs and Border Protection (CBP). The I-94 provides a count of visitor arrivals to the United States (with stays of 1-night or more and visiting under certain visa types) to calculate U.S. travel and tourism volume exports.
- **U.S. City Demographic Data:** This data comes from OpenSoft. You can read more about it **[here](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/)**.
- **Airport Code Table:** This is a simple table of airport codes and corresponding cities. It comes from **[here](https://datahub.io/core/airport-codes#data)**.
- **World Temperature Data**: 

The main tools which are going to be used are:
- **Python libraries** like pandas or numpy
- **Pyspark** to deal with the immigration dataset
- **Aparhe Airflow** to automate a pipeline to extract this information programatically and to mantain the database updated.
- **Amazon S3**: to store both the raw data and the final data lake in parquet.
- **Amazon EMR**: to process the data with PySpark.

#### DATA DESCRIPTION

- **I94 Immigration Data:** This is a dataset with information from the people which arrive to EEUU as immigrants. 

- **U.S. City Demographic Data:** The information included in this dataset is the following:
    - **City names**
    - **State**: 
    - **Median age**
    - **Male population**
    - **Female population**
    - **Total population**
    - **Number of veterans**
    - **Foreign born**
    - **Average household size**
    - **State code**
    - **Race**
    - **Statistic values**
    
    
- **Airport Code Table:** Dataset with information about different airports. This information includes:
    - **ident**: Identification code
    - **type**: type of airport
    - **name**: name of the airport
    - **elevation_ft**: elevation above the sea level
    - **iso_country**: iso code of each country
    - **iso_region**: iso code of each region
    - **municipality**: municipality where the airport is located
    - **gps_code**: gps code of the airport
    - **iata_code**: An IATA airport code, also known as an IATA location identifier, IATA station code, or simply a location identifier, is a three-character alphanumeric geocode designating many airports and metropolitan areas around the world, defined by the International Air Transport Association (IATA).
    - **local_code**: local code of the airport
    - **coordinates**: coordinates of the airport
    
    
- **Global land temperatures by city**: Dataset with informmation about the temperature in different cities at different dates.
    - **dt**: date of the data
    - **AverageTemperature**
    - **AverageTemperatureUncertainty**
    - **City**
    - **Country**
    - **Latitude**
    - **Longitude**

**I94 Immigration Data**

In [4]:
# df_sas = spark.read.parquet('sas_data/*.parquet')

In [5]:
def read_immigration_data(url = 'immigration_data_sample.csv'):
    '''
    Function which loads the immigration dataset.
    
    INPUT:
    url (string): URL of the bucket where the information is stored.
    
    OUTPUT:
    df_sas (Spark DataFrame): dataframe created based on the data
    '''
    df_sas = spark.read.csv('immigration_data_sample.csv', header = True)
#     df_sas = spark.read.parquet('sas_data/*.parquet'
    return df_sas

In [6]:
df_sas = read_immigration_data() #OK

**U.S. City Demographic Data**

In [7]:
def read_cities_data(url = 'us-cities-demographics.csv'):
    '''
    Function which loads the immigration dataset.
    
    INPUT:
    url (string): URL of the bucket where the information is stored.
    
    OUTPUT:
    df_sas (Spark DataFrame): dataframe created based on the data
    '''
    df_cities = pd.read_csv('us-cities-demographics.csv', sep=';')
    return df_cities

# df_cities = pd.read_csv('us-cities-demographics.csv', sep=';')

In [8]:
df_cities = read_cities_data() #OK

**Airport Code Table**

In [9]:
def read_cities_data(url = 'airport-codes_csv.csv'):
    '''
    Function which loads the immigration dataset.
    
    INPUT:
    url (string): URL of the bucket where the information is stored.
    
    OUTPUT:
    df_sas (Spark DataFrame): dataframe created based on the data
    '''
    df_airport = pd.read_csv(url)
    return df_airport

# df_airport = pd.read_csv('airport-codes_csv.csv')

In [10]:
df_airport = read_cities_data() #OK

**Global land temperatures by city**

In [11]:
def read_temp_data(url = "C:/Users/gonza/Downloads/GlobalLandTemperaturesByCity.csv"):
    '''
    Function which loads the immigration dataset.
    
    INPUT:
    url (string): URL of the bucket where the information is stored.
    
    OUTPUT:
    df_sas (Spark DataFrame): dataframe created based on the data
    '''
    df_temp = spark.read.csv(url, header = True, inferSchema = True)
    return df_temp

In [12]:
df_temp = read_temp_data() #OK, pero el toPandas no funciona y no entiendo el por qué

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Although each file and dataset is different and have different problems to solve but the steps to be implemented are the following:

* Modify the name of the columns to more descriptive names
* Modify data types
* Fox the missing values
* Drop duplicates values
* Replace codes with more descriptive names
* Drop unnecesary columns

Not all steps will have to be applied to all datasets.


### 1. I94 Immigration Data

#### Explore the data

In [13]:
df_sas.limit(6).toPandas()

,_c0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,None,M,1955.0,07202016,F,None,JL,56582674633.0,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,None,M,1990.0,10222016,M,None,*GA,94361995930.0,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,None,M,1940.0,07052016,M,None,LH,55780468433.0,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,None,M,1991.0,10272016,M,None,QR,94789696030.0,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,None,M,1997.0,07042016,F,None,None,42322572633.0,LAND,WT
5,721257,1481650.0,2016.0,4.0,577.0,577.0,ATL,20552.0,1.0,GA,...,None,M,1965.0,10072016,M,None,DL,736852585.0,910,B2


In [14]:
df_sas.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- cicid: string (nullable = true)
 |-- i94yr: string (nullable = true)
 |-- i94mon: string (nullable = true)
 |-- i94cit: string (nullable = true)
 |-- i94res: string (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: string (nullable = true)
 |-- i94mode: string (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: string (nullable = true)
 |-- i94bir: string (nullable = true)
 |-- i94visa: string (nullable = true)
 |-- count: string (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: string (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = tru

In [15]:
print((df_sas.count(), len(df_sas.columns)))

(1000, 29)


In [16]:
df_sas.summary().toPandas()

,summary,_c0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,count,1000,1000,1000,1000,1000,1000,1000,1000,1000,...,0,954,1000,1000,859,35,967,1000,992,1000
1,mean,1542097.12,3040461.409,2016.0,4.0,302.928,298.262,None,20559.68,1.078,...,None,None,1973.618,8258277.404255319,None,3826.8571428571427,2.0,6.9372367950789E10,1337.2554291623578,None
2,stddev,915287.9043923795,1799817.7827726966,0.0,0.0,206.48528516334758,202.12038988683958,None,8.995026987758733,0.4859548869516101,...,None,None,17.90342449389525,1622586.3557888167,None,221.7425829858661,0.0,2.338134181802248E10,6149.954574383991,None
3,min,1006205,1000074.0,2016.0,4.0,103.0,103.0,AGA,20545.0,1.0,...,None,M,1923.0,04082018,F,3468,*GA,0.0,00001,B1
4,25%,721257.0,1408683.0,2016.0,4.0,135.0,131.0,None,20552.0,1.0,...,None,None,1961.0,7092016.0,None,3668.0,2.0,5.5991897633E10,100.0,None
5,50%,1494106.0,2938927.0,2016.0,4.0,213.0,213.0,None,20560.0,1.0,...,None,None,1974.0,7252016.0,None,3887.0,2.0,5.9313924133E10,410.0,None
6,75%,2360660.0,4693164.0,2016.0,4.0,438.0,438.0,None,20567.0,1.0,...,None,None,1985.0,1.0122016E7,None,3943.0,2.0,9.343458123E10,906.0,None
7,max,997880,999282.0,2016.0,4.0,746.0,696.0,X96,20574.0,9.0,...,None,M,2015.0,D/S,X,4686,ZX,95021509030.0,XBLNG,WT


In [17]:
# Missing values analysis
df_sas.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_sas.columns]).take(1)

[Row(_c0=0, cicid=0, i94yr=0, i94mon=0, i94cit=0, i94res=0, i94port=0, arrdate=0, i94mode=0, i94addr=59, depdate=49, i94bir=0, i94visa=0, count=0, dtadfile=0, visapost=618, occup=996, entdepa=0, entdepd=46, entdepu=1000, matflag=46, biryear=0, dtaddto=0, gender=141, insnum=965, airline=33, admnum=0, fltno=8, visatype=0)]

There are missing values in the variables `i94addr=59`, `depdate=49`, `visapost=618`, `occup=996`, `entdepd=46`, `entdepu=1000`, `matflag=46`, `gender=141`, `insnum=965`, `airline=33` and `fltno=8`. It is going to be necessary to analyze each of them to identify a strategy to avoid having missing values, if possible.

#### Cleaning steps

In [18]:
def cleaning_immigration_data(df_sas):
    '''
    Function which cleans the data implementing the following steps: 
    1. Change column names
    2. Change the data types
    3. Fix the missing values
    4. Drop duplicate values
    5. Replace codes with more descriptive values
    6. Drop unnecesary columns
    
    INPUT:
    df_sas (Spark DataFrame): DataFrame with
    
    OUTPUT:
    df_sas (Spark DataFrame): data already cleaned for being uploaded to the data lake
    '''
    
    # 1. Change column names
    # Keys to modify the codes for names easier to understand
    names = {'cicid':'immigrant_id','I94YR': 'year', 'I94MON':'month', 'I94CIT':'country_1','I94RES':'country_2','I94PORT':'city',
        'ARRDATE':'arrival_date','I94MODE':'transport_mode','I94ADDR':'state','DEPDATE':'departure_date',
         'I94BIR':'age_respondent','I94VISA':'visa_code','COUNT':'summary_statistics','DTADFILE':'character_date_field',
        'VISAPOST':'department_visa','OCCUP':'occupation','ENTDEPA':'arrival_flag','ENTDEPD':'departure_flag',
        'ENTDEPU':'update_flag','MATFLAG':'match_flag','BIRYEAR':'birth_year','DTADDTO':'character_date_field',
        'GENDER':'non_inmigrant_sex','INSNUM':'ins_number','AIRLINE':'airline','ADMNUM':'admission_number',
        'FLTNO':'flight_number','VISATYPE':'visa_type'}
    
    # Modification of the column names for others more intuitive
    for i in names:
        df_sas = df_sas.withColumnRenamed(i,names[i])

    # 2. Change the data types
    from pyspark.sql.functions import substring, length, col, expr
    df_sas = df_sas.withColumn("country_1",expr("substring(country_1, 1, length(country_1)-2)"))
    df_sas = df_sas.withColumn("immigrant_id",expr("substring(immigrant_id, 1, length(immigrant_id)-2)"))
    df_sas = df_sas.withColumn("year",expr("substring(year, 1, length(year)-2)"))
    df_sas = df_sas.withColumn("month",expr("substring(month, 1, length(month)-2)"))
    df_sas = df_sas.withColumn("country_2",expr("substring(country_2, 1, length(country_2)-2)"))
    df_sas = df_sas.withColumn("arrival_date",expr("substring(arrival_date, 1, length(arrival_date)-2)"))
    df_sas = df_sas.withColumn("transport_mode",expr("substring(transport_mode, 1, length(transport_mode)-2)"))
    df_sas = df_sas.withColumn("departure_date",expr("substring(departure_date, 1, length(departure_date)-2)"))
    df_sas = df_sas.withColumn("age_respondent",expr("substring(age_respondent, 1, length(age_respondent)-2)"))
    df_sas = df_sas.withColumn("visa_code",expr("substring(visa_code, 1, length(visa_code)-2)"))
    df_sas = df_sas.withColumn("summary_statistics",expr("substring(summary_statistics, 1, length(summary_statistics)-2)"))
    df_sas = df_sas.withColumn("birth_year",expr("substring(birth_year, 1, length(birth_year)-2)"))
    df_sas = df_sas.withColumn("admission_number",expr("substring(admission_number, 1, length(admission_number)-2)"))
    
    df_sas = df_sas.withColumn("country_1", col("country_1").cast('int'))
    df_sas = df_sas.withColumn("immigrant_id", col("immigrant_id").cast('int'))
    df_sas = df_sas.withColumn("year", col("year").cast('int'))
    df_sas = df_sas.withColumn("month", col("month").cast('int'))
    df_sas = df_sas.withColumn("country_2", col("country_2").cast('int'))
    df_sas = df_sas.withColumn("arrival_date", col("arrival_date").cast('int'))
    df_sas = df_sas.withColumn("transport_mode", col("transport_mode").cast('int'))
    df_sas = df_sas.withColumn("departure_date", col("departure_date").cast('int'))
    df_sas = df_sas.withColumn("age_respondent", col("age_respondent").cast('int'))
    df_sas = df_sas.withColumn("visa_code", col("visa_code").cast('int'))
    df_sas = df_sas.withColumn("summary_statistics", col("summary_statistics").cast('int'))
    df_sas = df_sas.withColumn("birth_year", col("birth_year").cast('int'))
    df_sas = df_sas.withColumn("admission_number", col("admission_number").cast('int'))
    
    # 3. Fix the missing values
    # 3.1. I94ADDR (state)
    df_sas = df_sas.filter(df_sas.state.isNotNull())
    
    # 3.2. DEPDATE (departure_date)
    df_sas.filter(df_sas.departure_date.isNotNull()).toPandas()
    
    # 3.3. VISAPOST (department_visa)
    df_sas = df_sas.drop('department_visa')
    
    # 3.4. OCCUP (occupation)
    df_sas = df_sas.drop('occupation')
    
    # 3.5. ENTDEPD (departure_flag)
    df_sas = df_sas.filter(df_sas.departure_flag.isNotNull())
    
    # 3.6. MATFLAG (match_flag)
    # There are no missing values anymore, because those ones has been already solved in a prior step.    
    
    # 4. Drop duplicate values
    df_sas = df_sas.dropDuplicates()    
    
    # 5. Replace codes with more descriptive values
    # Read the SAS file with the meaning of the codes of the columns I94CIT & I94RES,I94PORT, I94MODE, I94ADDR
    country_codes = 'I94_SAS_Labels_Descriptions.SAS'
    with open(country_codes) as f:
        lines = f.readlines()
    lines = [line.replace('"','').replace('\n','').replace("'",'') for line in lines]
    
    df_sas_codes = pd.DataFrame(lines)
    
    # 5.1. I94CIT & I94RES
    values_I94CIT_I94RES = df_sas_codes[9:298]
    values_I94CIT_I94RES = values_I94CIT_I94RES[0].str.split('=', expand = True)
    values_I94CIT_I94RES.rename(columns = {0:"code" , 1:"name"}, inplace = True)
    values_I94CIT_I94RES = spark.createDataFrame(values_I94CIT_I94RES)
    df_sas = df_sas.join(values_I94CIT_I94RES, df_sas.country_1 == values_I94CIT_I94RES.code)
    df_sas = df_sas.drop('code').withColumnRenamed('name','country_1_name')
    df_sas = df_sas.join(values_I94CIT_I94RES, df_sas.country_2 == values_I94CIT_I94RES.code)
    df_sas = df_sas.drop('code').withColumnRenamed('name','country_2_name')
    
    # 5.2. I94PORT
    values_I94PORT = df_sas_codes[302:962]
    values_I94PORT = values_I94PORT[0].str.split('=', expand = True)
    values_I94PORT.rename(columns = {0:"code" , 1:"name"}, inplace = True)
    values_I94PORT['code'] = values_I94PORT['code'].str.strip()
    values_I94PORT['name'] = values_I94PORT['name'].str.strip()
    values_I94PORT.rename(columns = {0:"code" , 1:"name"}, inplace = True)
    values_I94PORT = spark.createDataFrame(values_I94PORT)
    df_sas = df_sas.join(values_I94PORT, df_sas.city == values_I94PORT.code)
    df_sas = df_sas.drop('code').withColumnRenamed('name','city_name')    
    
    # 5.3. I94MODE
    values_I94MODE = df_sas_codes[972:976]
    values_I94MODE = values_I94MODE[0].str.split('=', expand = True)
    values_I94MODE.rename(columns = {0:"code" , 1:"name"}, inplace = True)
    values_I94MODE['code'] = values_I94MODE['code'].str.strip()
    values_I94MODE['name'] = values_I94MODE['name'].str.strip()
    values_I94MODE = spark.createDataFrame(values_I94MODE)
    df_sas = df_sas.join(values_I94MODE, df_sas.transport_mode == values_I94MODE.code)
    df_sas = df_sas.drop('code').withColumnRenamed('name','transport_mode_name')
    
    # 5.4. I94ADDR
    values_I94ADDR= df_sas_codes[982:1036]
    values_I94ADDR = values_I94ADDR[0].str.split('=', expand = True)
    values_I94ADDR.rename(columns = {0:"code" , 1:"name"}, inplace = True)
    values_I94ADDR['code'] = values_I94ADDR['code'].str.strip()
    values_I94ADDR['name'] = values_I94ADDR['name'].str.strip()
    values_I94ADDR = spark.createDataFrame(values_I94ADDR)
    df_sas = df_sas.join(values_I94ADDR, df_sas.state == values_I94ADDR.code)
    df_sas = df_sas.drop('code').withColumnRenamed('name', 'state_name')
    
    
    # 6. Drop unnecesary columns
    df_sas = df_sas.drop('_c0')
    
    return df_sas
    

In [19]:
df_sas = read_immigration_data() #OK

In [20]:
df_sas = cleaning_immigration_data(df_sas) #No funciona, devuelve un dataframe vacio.

In [21]:
df_sas.limit(5).toPandas()

,immigrant_id,year,month,country_1,country_2,city,arrival_date,transport_mode,state,departure_date,...,ins_number,airline,admission_number,flight_number,visa_type,country_1_name,country_2_name,city_name,transport_mode_name,state_name
0,5365953,2016,4,582,582,HOU,20572,1,PA,20581,...,None,UA,NaN,04429,B2,"MEXICO Air Sea, and Not Reported (I-94, no l...","MEXICO Air Sea, and Not Reported (I-94, no l...","HOUSTON, TX",Air,PENNSYLVANIA
1,4800385,2016,4,582,582,ATL,20569,1,MI,20611,...,None,DL,NaN,00331,B1,"MEXICO Air Sea, and Not Reported (I-94, no l...","MEXICO Air Sea, and Not Reported (I-94, no l...","ATLANTA, GA",Air,MICHIGAN
2,5358837,2016,4,582,582,NYC,20572,1,TX,20575,...,None,AM,NaN,00418,B2,"MEXICO Air Sea, and Not Reported (I-94, no l...","MEXICO Air Sea, and Not Reported (I-94, no l...","NEW YORK, NY",Air,TEXAS
3,930868,2016,4,582,582,WAS,20549,1,DC,20552,...,None,UA,NaN,01567,B2,"MEXICO Air Sea, and Not Reported (I-94, no l...","MEXICO Air Sea, and Not Reported (I-94, no l...",WASHINGTON DC,Air,DIST. OF COLUMBIA
4,2825582,2016,4,582,582,MIA,20559,1,FL,20565,...,None,AM,NaN,00420,B2,"MEXICO Air Sea, and Not Reported (I-94, no l...","MEXICO Air Sea, and Not Reported (I-94, no l...","MIAMI, FL",Air,FLORIDA


#### Upload data to the data lake

In [ ]:
df_sas.write.partitionBy('year').mode('overwrite').parquet(output_data + immigration/)

---
---

### 2. Airport Code Tables

#### 2.1. Explore the data

First, let's take a look at the code structure of this data:

In [22]:
df_airport = pd.read_csv('airport-codes_csv.csv')
df_airport.head(5)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [23]:
df_airport.shape

(55075, 12)

In [24]:
df_airport.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55075 entries, 0 to 55074
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ident         55075 non-null  object 
 1   type          55075 non-null  object 
 2   name          55075 non-null  object 
 3   elevation_ft  48069 non-null  float64
 4   continent     27356 non-null  object 
 5   iso_country   54828 non-null  object 
 6   iso_region    55075 non-null  object 
 7   municipality  49399 non-null  object 
 8   gps_code      41030 non-null  object 
 9   iata_code     9189 non-null   object 
 10  local_code    28686 non-null  object 
 11  coordinates   55075 non-null  object 
dtypes: float64(1), object(11)
memory usage: 5.0+ MB


In [25]:
df_airport.describe(include = 'all')

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
count,55075,55075,55075,48069.000000,27356,54828,55075,49399,41030,9189,28686,55075
unique,55075,7,52144,NaN,6,243,2810,27133,40850,9042,27436,54874
top,W34,small_airport,Centre Hospitalier Heliport,NaN,EU,US,US-TX,Seoul,MBAC,0,LAN,"0, 0"
freq,1,33965,85,NaN,7840,22757,2277,404,3,80,5,53
mean,NaN,NaN,NaN,1240.789677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,1602.363459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,-1266.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,205.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,718.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,1497.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There are a variables with a high amount of missing values, so it is going to be necessary to analyze, if they have to be removed or those values can be filled in.

In [26]:
df_airport.isna().sum()

ident               0
type                0
name                0
elevation_ft     7006
continent       27719
iso_country       247
iso_region          0
municipality     5676
gps_code        14045
iata_code       45886
local_code      26389
coordinates         0
dtype: int64

In [27]:
df_airport.isna().sum()/df_airport.shape[0]*100

ident            0.000000
type             0.000000
name             0.000000
elevation_ft    12.720835
continent       50.329551
iso_country      0.448479
iso_region       0.000000
municipality    10.305946
gps_code        25.501589
iata_code       83.315479
local_code      47.914662
coordinates      0.000000
dtype: float64

In [28]:
df_airport[df_airport.iata_code.isna()]

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"
...,...,...,...,...,...,...,...,...,...,...,...,...
55065,ZYTH,small_airport,Tahe Airport,1240.0,AS,CN,CN-23,Tahe,ZYTH,NaN,NaN,"124.720222222, 52.2244444444"
55071,ZYYY,medium_airport,Shenyang Dongta Airport,NaN,AS,CN,CN-21,Shenyang,ZYYY,NaN,NaN,"123.49600219726562, 41.784400939941406"
55072,ZZ-0001,heliport,Sealand Helipad,40.0,EU,GB,GB-ENG,Sealand,NaN,NaN,NaN,"1.4825, 51.894444"
55073,ZZ-0002,small_airport,Glorioso Islands Airstrip,11.0,AF,TF,TF-U-A,Grande Glorieuse,NaN,NaN,NaN,"47.296388888900005, -11.584277777799999"


There are no duplicated row.

In [29]:
df_airport[df_airport.duplicated()]

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates


#### Cleaning steps

In [30]:
def cleaning_airport_data(df_airport):
    '''
    Function which cleans the data implementing the following steps: 
    1. Modify the names of the columns to more descriptive values
    2. Drop unnecessary columns
    3. Modify data types
    4. Replace codes with more descriptive values
    5. Drop duplicated values
    
    INPUT:
    df_airport (Spark DataFrame): DataFrame with
    
    OUTPUT:
    df_airport (Spark DataFrame): data already cleaned for being uploaded to the data lake
    '''
    
    # 1. Modify the names of the columns to more descriptive values
    airport_coordinates = df_airport['coordinates'].str.split(',', expand = True).rename(columns={0:'latitude', 1:'longitude'})
    df_airport['latitude'] = airport_coordinates['latitude']
    df_airport['longitude'] = airport_coordinates['longitude']
    df_airport.drop('coordinates', axis = 1, inplace = True)
    
    # 2. Drop unnecessary columns
    df_airport.drop(['iata_code','continent', 'gps_code', 'local_code', 'elevation_ft', 'municipality',
                 'iso_country'], axis = 1, inplace = True)
    
    # 3. Modify data types
    df_airport.latitude = df_airport.latitude.astype('float').round(2)
    df_airport.longitude = df_airport.longitude.astype('float').round(2)
    df_iso_states = pd.read_csv('ISO_code_US.csv', sep=";")

    # 4. Replace codes with more descriptive values
    df_iso_states = pd.read_csv('ISO_code_US.csv', sep=";")
    df_airport = pd.merge(df_airport, df_iso_states, how = 'inner', left_on=['iso_region'], right_on = ['iso_code'])
    df_airport.drop("iso_code", axis = 1, inplace = True)
    
    # 5. Drop duplicated values
    df_airport = df_airport.drop_duplicates()
    
    return df_airport

In [31]:
df_airport = cleaning_airport_data(df_airport)
df_airport.head()

,ident,type,name,iso_region,latitude,longitude,state
0,00A,heliport,Total Rf Heliport,US-PA,-74.93,40.07,Pennsylvania
1,00PA,heliport,R J D Heliport,US-PA,-75.75,39.95,Pennsylvania
2,00PN,small_airport,Ferrell Field,US-PA,-80.21,41.30,Pennsylvania
3,00PS,closed,Thomas Field,US-PA,-77.37,40.38,Pennsylvania
4,01PA,heliport,Pine Heliport,US-PA,-80.05,40.66,Pennsylvania


#### Upload data to the data lake

In [ ]:
df_airport.to_csv(output_data + airport/)

---
---

### 3. U.S. Cities Demographic Data

#### Explore the data

In [32]:
# U.S. City Demographic Data: This data comes from OpenSoft.
df_cities = pd.read_csv('us-cities-demographics.csv', sep=';')
df_cities.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [33]:
df_cities.shape

(2891, 12)

In [34]:
df_cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   City                    2891 non-null   object 
 1   State                   2891 non-null   object 
 2   Median Age              2891 non-null   float64
 3   Male Population         2888 non-null   float64
 4   Female Population       2888 non-null   float64
 5   Total Population        2891 non-null   int64  
 6   Number of Veterans      2878 non-null   float64
 7   Foreign-born            2878 non-null   float64
 8   Average Household Size  2875 non-null   float64
 9   State Code              2891 non-null   object 
 10  Race                    2891 non-null   object 
 11  Count                   2891 non-null   int64  
dtypes: float64(6), int64(2), object(4)
memory usage: 271.2+ KB


In [35]:
df_cities.describe(include = 'all')

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
count,2891,2891,2891.000000,2.888000e+03,2.888000e+03,2.891000e+03,2878.000000,2.878000e+03,2875.000000,2891,2891,2.891000e+03
unique,567,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49,5,NaN
top,Columbia,California,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA,Hispanic or Latino,NaN
freq,15,676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,676,596,NaN
mean,NaN,NaN,35.494881,9.732843e+04,1.017696e+05,1.989668e+05,9367.832523,4.065360e+04,2.742543,NaN,NaN,4.896377e+04
std,NaN,NaN,4.401617,2.162999e+05,2.315646e+05,4.475559e+05,13211.219924,1.557491e+05,0.433291,NaN,NaN,1.443856e+05
min,NaN,NaN,22.900000,2.928100e+04,2.734800e+04,6.321500e+04,416.000000,8.610000e+02,2.000000,NaN,NaN,9.800000e+01
25%,NaN,NaN,32.800000,3.928900e+04,4.122700e+04,8.042900e+04,3739.000000,9.224000e+03,2.430000,NaN,NaN,3.435000e+03
50%,NaN,NaN,35.300000,5.234100e+04,5.380900e+04,1.067820e+05,5397.000000,1.882200e+04,2.650000,NaN,NaN,1.378000e+04
75%,NaN,NaN,38.000000,8.664175e+04,8.960400e+04,1.752320e+05,9368.000000,3.397175e+04,2.950000,NaN,NaN,5.444700e+04


In [36]:
df_cities.isna().sum()

City                       0
State                      0
Median Age                 0
Male Population            3
Female Population          3
Total Population           0
Number of Veterans        13
Foreign-born              13
Average Household Size    16
State Code                 0
Race                       0
Count                      0
dtype: int64

In [37]:
df_cities[df_cities['Number of Veterans'].isna()]

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
111,San Juan,Puerto Rico,41.4,155408.0,186829.0,342237,NaN,NaN,NaN,PR,Hispanic or Latino,335559
155,Caguas,Puerto Rico,40.4,34743.0,42265.0,77008,NaN,NaN,NaN,PR,Hispanic or Latino,76349
258,Carolina,Puerto Rico,42.0,64758.0,77308.0,142066,NaN,NaN,NaN,PR,American Indian and Alaska Native,12143
637,Carolina,Puerto Rico,42.0,64758.0,77308.0,142066,NaN,NaN,NaN,PR,Hispanic or Latino,139967
1747,San Juan,Puerto Rico,41.4,155408.0,186829.0,342237,NaN,NaN,NaN,PR,American Indian and Alaska Native,4031
1748,Mayagüez,Puerto Rico,38.1,30799.0,35782.0,66581,NaN,NaN,NaN,PR,Asian,235
1995,Ponce,Puerto Rico,40.5,56968.0,64615.0,121583,NaN,NaN,NaN,PR,Hispanic or Latino,120705
2004,Bayamón,Puerto Rico,39.4,80128.0,90131.0,170259,NaN,NaN,NaN,PR,Hispanic or Latino,169155
2441,San Juan,Puerto Rico,41.4,155408.0,186829.0,342237,NaN,NaN,NaN,PR,Asian,2452
2589,Guaynabo,Puerto Rico,42.2,33066.0,37426.0,70492,NaN,NaN,NaN,PR,Hispanic or Latino,69936


Todos los valores perdidos de Number of Veterans y Foreignborn son de Puerto Rico

In [38]:
df_cities[df_cities['Average Household Size'].isna()]

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
111,San Juan,Puerto Rico,41.4,155408.0,186829.0,342237,NaN,NaN,NaN,PR,Hispanic or Latino,335559
155,Caguas,Puerto Rico,40.4,34743.0,42265.0,77008,NaN,NaN,NaN,PR,Hispanic or Latino,76349
258,Carolina,Puerto Rico,42.0,64758.0,77308.0,142066,NaN,NaN,NaN,PR,American Indian and Alaska Native,12143
333,The Villages,Florida,70.5,NaN,NaN,72590,15231.0,4034.0,NaN,FL,Hispanic or Latino,1066
449,The Villages,Florida,70.5,NaN,NaN,72590,15231.0,4034.0,NaN,FL,Black or African-American,331
637,Carolina,Puerto Rico,42.0,64758.0,77308.0,142066,NaN,NaN,NaN,PR,Hispanic or Latino,139967
1437,The Villages,Florida,70.5,NaN,NaN,72590,15231.0,4034.0,NaN,FL,White,72211
1747,San Juan,Puerto Rico,41.4,155408.0,186829.0,342237,NaN,NaN,NaN,PR,American Indian and Alaska Native,4031
1748,Mayagüez,Puerto Rico,38.1,30799.0,35782.0,66581,NaN,NaN,NaN,PR,Asian,235
1995,Ponce,Puerto Rico,40.5,56968.0,64615.0,121583,NaN,NaN,NaN,PR,Hispanic or Latino,120705


In [39]:
df_cities[df_cities.duplicated()]

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count


#### Cleaning steps

In [40]:
def cleaning_cities_data(df_cities):
    '''
    Function which cleans the data implementing the following steps: 
    1. Modify the names of the columns to more descriptive values
    2. Fix the missing values issues
    3. Modify data types
    
    INPUT:
    df_cities (Spark DataFrame): DataFrame directly extracted from the raw data without any cleaning process
    
    OUTPUT:
    df_cities (Spark DataFrame): data already cleaned for being uploaded to the data lake
    '''
    
    # 1. Modify the names of the columns to more descriptive values
    df_cities.columns = df_cities.columns.str.lower()
    df_cities.columns = df_cities.columns.str.replace(" ","_")
    df_cities.columns = df_cities.columns.str.replace("foreign-born","foreign_born")
    
    # 2. Fix the missing values issues
    # median age
    list_states = list(df_cities.state.unique())
    for i in list_states:
        if df_cities[df_cities.state==i]['median_age'].mean()>0:
            df_cities['median_age'] = df_cities.groupby('state')['median_age'].transform(lambda x: x.fillna(x.mean()))

        else:
            df_cities['median_age'] = df_cities.groupby('state')['median_age'].transform(lambda x: x.fillna(df_cities.median_age.mean()))
            
    # male population
    list_states = list(df_cities.state.unique())
    for i in list_states:
        if df_cities[df_cities.state==i]['male_population'].mean()>0:
            df_cities['male_population'] = df_cities.groupby('state')['male_population'].transform(lambda x: x.fillna(x.mean()))

        else:
            df_cities.male_population = df_cities.groupby('state')['male_population'].transform(lambda x: x.fillna(df_cities.male_population.mean()))
            
    # female population
    list_states = list(df_cities.state.unique())
    for i in list_states:
        if df_cities[df_cities.state==i]['female_population'].mean()>0:
            df_cities['female_population'] = df_cities.groupby('state')['female_population'].transform(lambda x: x.fillna(x.mean()))

        else:
            df_cities.male_population = df_cities.groupby('state')['female_population'].transform(lambda x: x.fillna(df_cities.female_population.mean()))
    
    # total population
    list_states = list(df_cities.state.unique())
    for i in list_states:
        if df_cities[df_cities.state==i]['total_population'].mean()>0:
            df_cities['total_population'] = df_cities.groupby('state')['total_population'].transform(lambda x: x.fillna(x.mean()))

        else:
            df_cities['total_population'] = df_cities.groupby('state')['total_population'].transform(lambda x: x.fillna(df_cities.total_population.mean()))    
            
    # number of veterans
    list_states = list(df_cities.state.unique())
    for i in list_states:
        if df_cities[df_cities.state==i]['number_of_veterans'].mean()>0:
            df_cities['number_of_veterans'] = df_cities.groupby('state')['number_of_veterans'].transform(lambda x: x.fillna(x.mean()))

        else:
            df_cities['number_of_veterans'] = df_cities.groupby('state')['number_of_veterans'].transform(lambda x: x.fillna(df_cities.number_of_veterans.mean()))
    
    # foreign born
    list_states = list(df_cities.state.unique())
    for i in list_states:
        if df_cities[df_cities.state==i]['foreign_born'].mean()>0:
            df_cities['foreign_born'] = df_cities.groupby('state')['foreign_born'].transform(lambda x: x.fillna(x.mean()))

        else:
            df_cities['foreign_born'] = df_cities.groupby('state')['foreign_born'].transform(lambda x: x.fillna(df_cities.foreign_born.mean()))
    
    # average household size
    list_states = list(df_cities.state.unique())
    for i in list_states:
        if df_cities[df_cities.state==i]['average_household_size'].mean()>0:
            df_cities['average_household_size'] = df_cities.groupby('state')['average_household_size'].transform(lambda x: x.fillna(x.mean()))

        else:
            df_cities.average_household_size = df_cities.groupby('state')['average_household_size'].transform(lambda x: x.fillna(df_cities.average_household_size.mean()))
    
    # count
    list_states = list(df_cities.state.unique())
    for i in list_states:
        if df_cities[df_cities.state==i]['count'].mean()>0:
            df_cities['count'] = df_cities.groupby('state')['count'].transform(lambda x: x.fillna(x.mean()))

        else:
            df_cities['count'] = df_cities.groupby('state')['count'].transform(lambda x: x.fillna(df_cities.count.mean()))
    
    # Drop the lines of the columns which have key values that can not be infered
    df_cities.dropna(subset = ['city','state','state_code','race'], inplace = True)
    
    # 3. Modify data types
    list_float_to_int = ['male_population','female_population','total_population','number_of_veterans',
                    'foreign_born','count']

    for col in list_float_to_int:
        df_cities[col] = df_cities[col].astype('int')

    # 4. Replace codes with more descriptive values
    
    
    # 5. Drop duplicated values
    
    return df_cities

In [41]:
df_cities = cleaning_cities_data(df_cities)
df_cities.head()

,city,state,median_age,male_population,female_population,total_population,number_of_veterans,foreign_born,average_household_size,state_code,race,count
0,Silver Spring,Maryland,33.8,40601,41862,82463,1562,30908,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129,49500,93629,4147,32935,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040,46799,84839,4819,8229,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127,87105,175232,5821,33878,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040,143873,281913,5829,86253,2.73,NJ,White,76402


#### Upload data to the data lake

In [ ]:
df_cities.to_csv(output_data + immigration/)

In [45]:
df_cities.to_csv(output_data)

ValueError: 

---
---

### 4. World Temperature Data

#### Explore the data

In [ ]:
df_temp = spark.read.csv("C:/Users/gonza/Downloads/GlobalLandTemperaturesByCity.csv", header = True, inferSchema = True)

In [ ]:
df_temp_eeuu = df_temp.where(df_temp.Country == "United States")

In [ ]:
df_temp_eeuu.show(5)

In [ ]:
df_temp_eeuu.printSchema()

In [ ]:
df_temp_eeuu.describe().show()

#### Cleaning steps

In [ ]:
def cleaning_temperature_data(df_temp):
    '''
    Function which cleans the data implementing the following steps: 
    1. Modify the data types
    2. Fix the missing values issues
    3. Drop unnecessary columns
    
    INPUT:
    df_temp (Spark DataFrame): DataFrame directly extracted from the raw data without any cleaning process
    
    OUTPUT:
    df_temp_eeuu (Spark DataFrame): data already cleaned for being uploaded to the data lake
    '''
    df_temp_eeuu = df_temp.where(df_temp.Country == "United States")
    
    # 1. Modify the data types
    
    # Change the datastamp format, since the hour, minutes and seconds are not necessary and do not contribute to the final result
    df_temp_eeuu = df_temp_eeuu.withColumn('dt', to_date('dt'))
    
    # 2. Fix the missing values issues
    df_pandas = df_temp_eeuu.toPandas()
    df_pandas['AverageTemperature'] = df_pandas.groupby('City')['AverageTemperature'].transform(lambda x: x.fillna(x.mean()))
    df_pandas['AverageTemperatureUncertainty'] = df_pandas.groupby('City')['AverageTemperatureUncertainty'].transform(lambda x: x.fillna(x.mean()))
    
    # 3. Drop unnecessary columns
    df_pandas.drop("Country", axis = 1, inplace = True)
    
    return df_temp_eeuu

In [ ]:
df_temp = read_temp_data() #OK, pero el toPandas no funciona y no entiendo el por qué

In [ ]:
df_temp_eeuu = cleaning_temperature_data(df_temp)

In [ ]:
df_temp_eeuu.limit(5).toPandas()

In [ ]:
output_data = "s3a://bucket-test-udacity/"
df_temp_eeuu.write.partitionBy('dt').mode('overwrite').csv(output_data)

---
---
---

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

---
---
---

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.